![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [183]:
# Import necessary packages
import os
import pandas as pd
import torch
import evaluate
from openai import OpenAI

from transformers import logging, pipeline
logging.set_verbosity(logging.WARNING)

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification


# Sentiment Analysis of car reviews

In [184]:
reviews = pd.read_csv('car_reviews.csv', sep=';')
reviews.head()

#We want a model that classifies only NEGATIVE or POSITIVE to be able to compute the accuracy and f1
sentiment_class = pipeline(task="sentiment-analysis", model="siebert/sentiment-roberta-large-english")

summary = []
for review in reviews['Review']:
    summary.append(sentiment_class(review))

pred_label = []
label_score = []
for i in range(len(summary)):
    pred_label.append(summary[i][0]['label'].upper())
    label_score.append(summary[i][0]['score'])

reviews['pred_class'] = pred_label
reviews['conf_pred_score'] = label_score

reviews


Device set to use cpu


,Review,Class,pred_class,conf_pred_score
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE,POSITIVE,0.998915
1,The car is fine. It's a bit loud and not very ...,NEGATIVE,NEGATIVE,0.999505
2,"My first foreign car. Love it, I would buy ano...",POSITIVE,POSITIVE,0.998864
3,I've come across numerous reviews praising the...,NEGATIVE,NEGATIVE,0.999491
4,I've been dreaming of owning an SUV for quite ...,POSITIVE,POSITIVE,0.998922


### Sentiment Analysis Evaluation Accuracy/F1

In [185]:
label_map = {'NEGATIVE':0, 'POSITIVE':1}

accuracy = evaluate.load('accuracy')
f1 = evaluate.load('f1')

accuracy_score = accuracy.compute(references=reviews['Class'].map(label_map), predictions=reviews['pred_class'].map(label_map))
f1_score = f1.compute(references=reviews['Class'].map(label_map), predictions=reviews['pred_class'].map(label_map))

print(f'{accuracy_score}\n{f1_score}')

{'accuracy': 1.0}
{'f1': 1.0}


# English to Spanish translaion LLM

In [186]:
first_review = reviews.loc[0,'Review']
first_two_sentences = first_review.split('.')[:2]
first_two_sentences

#Translation model english to spanish
translation_llm = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn",
    tgt_lang="spa_Latn"
)
translations = []
for sentence in first_two_sentences:
    translations.append(translation_llm(sentence))

full_translation = translations[0][0]['translation_text']+'. '+translations[1][0]['translation_text']

print(full_translation)



Device set to use cpu


Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas de negocios y uso personal.


### Translation Evaluation 

In [187]:
meteor = evaluate.load('meteor')
bleu = evaluate.load('bleu')

# Read file once
with open('reference_translations.txt', 'r', encoding='utf-8') as f:
    text = f.read().strip()

# Split into sentences
references_split = [s.strip() for s in text.split('.') if s.strip()]

# Take first two
reference1 = references_split[0] + '.' if len(references_split) > 0 else ''
reference2 = references_split[1] + '.' if len(references_split) > 1 else ''

# Build references (list of list of refs)
references = [[reference1, reference2]]

meteor_score = meteor.compute(references=references, predictions=[full_translation])
bleu_score = bleu.compute(references=references, predictions=[full_translation])

print(f'meteor_score: {meteor_score['meteor']}\nbleu_score: {bleu_score['bleu']}')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


meteor_score: 0.8892857142857143
bleu_score: 0.5877038209148266


# Q&A Model

In [191]:
QA = pipeline(
    "question-answering",
    model="deepset/minilm-uncased-squad2",
)
question = 'What did he like about the brand?'

#2nd review

#The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. 
#The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. 
#Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. 
#The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. 
#Initially, I really liked what the brand is about: ride quality, reliability, etc. 
#But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.;NEGATIVE

context = reviews.loc[1, 'Review']
answer = QA(question=question, context=context)

print(f'\nTEXT: {context}\nQUESTION: {question}\nANSWER: {answer['answer']}')

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



TEXT: The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
QUESTION: What did he like about the brand?
ANSWER: ride quality, reliability


# Summarizing Model

In [ ]:
summarizer = pipeline(
    "summarization",
    model="huggingface-course/mt5-small-finetuned-amazon-en-es",
)

#last_review
last_review = reviews.iloc[-1]['Review']
summary = summarizer(last_review, max_length=55)
review_summary = summary[0]['summary_text']
print('Last Review Summary: ',review_summary)

#evalutation
rouge = evaluate.load('rouge')
rouge_score = rouge.compute(predictions=[review_summary], references=[last_review])
bleu_summary_score = bleu.compute(predictions=[review_summary], references=[last_review])

print(f'rouge_score: {rouge}\nbleu_score: {bleu_summary_score['bleu']}')

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=55) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Last Review Summary:  I am very satisfied overall. The Nissan Rogue provides me with the desired SUV experience without Burdening me with the desired SUV experience without Burdening me with an exorbitant financial commitment
rouge_score: EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value('string'), 'references': List(Value('string'))}, {'predictions': Value('string'), 'references': Value('string')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence base

In [ ]:
import os
from openai import OpenAI

HF_TOKEN = os.getenv("HF_TOKEN")


client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

completion = client.chat.completions.create(
    model="moonshotai/Kimi-K2-Instruct-0905",
    messages=[
        {
            "role": "user",
            "content": ""
        }
    ],
)

print(completion.choices[0].message)

import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="together",
    api_key=HF_TOKEN,
)

# output is a PIL.Image object
image = client.text_to_image(
    "How you imagine me?",
    model="black-forest-labs/FLUX.1-dev",
)
image

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable